In [1]:
using Pipe
using Iterators
using DataStructures

In [2]:
function loadphrases(filename)
    (@pipe readall(filename) |> split(_,'\n'))[1:end-1];
end

function load_phrases_and_embeddings(base_filename)
    phrases = loadphrases(base_filename * ".phrases.txt")
    ps = readcsv(base_filename * ".csv")
    @assert size(ps,1)==length(phrases)
    phrases,ps
end

load_phrases_and_embeddings (generic function with 1 method)

In [3]:
#phrase_range = 14380:16000  #old testerment14380:40069
#ps = readcsv("results/gutten.csv")[phrase_range,:]
#phrases = loadphrases("results/gutten.phrases.txt") [phrase_range]

In [4]:
phrases,ps = load_phrases_and_embeddings( "results/wikipedia_turtle")

(SubString{ASCIIString}["turtles are *UNKNOWN* of the order *UNKNOWN* *UNKNOWN* or *UNKNOWN*","turtle , *UNKNOWN* or *UNKNOWN*","the word *UNKNOWN* is popular among *UNKNOWN* , scientists , and conservationists working with these animals as a catch-all name for any member of the *UNKNOWN* *UNKNOWN* , which includes all turtles living and *UNKNOWN* , as well as their immediate ancestors .","anatomy and *UNKNOWN* largest living *UNKNOWN* is the *UNKNOWN* sea turtle *UNKNOWN* *UNKNOWN* *UNKNOWN* *UNKNOWN* , which reaches a shell length of","giant *UNKNOWN* of the *UNKNOWN* *UNKNOWN* , *UNKNOWN* , and others were relatively widely distributed around the world into prehistoric times , and are known to have existed in north and south *UNKNOWN* , *UNKNOWN* , and *UNKNOWN* .","they became *UNKNOWN* at the same time as the appearance of man , and it is assumed humans hunted them for food .","the only surviving giant *UNKNOWN* are on the *UNKNOWN* and *UNKNOWN* islands , and can grow to over","n

In [5]:
using PyCall
@pyimport sklearn
@pyimport sklearn.mixture as sklm
@pyimport sklearn.cluster as skl_cluster

In [6]:
function show_clusters(model,words, labels)
    clusters = DefaultDict(Int, Vector{String}, ()->String[])
    for ii in 1:length(words)
        word = words[ii]
        cluster =  labels[ii]
        push!(clusters[cluster], word)
    end
    
    for cluster in keys(clusters)
        println("** $cluster **")
        for word in clusters[cluster]
            println(" - ", word)
        end
        println()
    end
    clusters
end


show_clusters (generic function with 1 method)

In [7]:
using Distances
push!(LOAD_PATH, "../word-embeddings2")
import WordEmbeddings.cosine_sim

In [8]:
pps = ps'
pps_cosine_sim = Float64[WordEmbeddings.cosine_sim(pps[:,ii],pps[:,jj]) for ii in 1:size(pps,2), jj in 1:size(pps,2)];


In [9]:
affinity = pycall(pybuiltin("intern"),PyObject, "precomputed")
affprop = skl_cluster.AffinityPropagation(damping=0.6,affinity=affinity)

affprop[:fit_predict](pps_cosine_sim)
centers = affprop[:cluster_centers_indices_] + 1 #julia indexing
for phrase in phrases[centers]
    println(phrase)
end

turtle , *UNKNOWN* or *UNKNOWN*
turtles are divided into two groups according to how they withdraw their necks into their shells *UNKNOWN* something the ancestral *UNKNOWN* could not do *UNKNOWN* .
*UNKNOWN* turtles usually have *UNKNOWN* ridges for slicing through their prey .
instead , they breathe in two ways .
most *UNKNOWN* have a large , *UNKNOWN* shell that makes it difficult for predators to crush the shell between their *UNKNOWN* .
one of the few exceptions is the *UNKNOWN* *UNKNOWN* *UNKNOWN* , which has a flat , flexible shell that allows it to hide in rock *UNKNOWN* .
*UNKNOWN* snapping turtles and *UNKNOWN* turtles have small , *UNKNOWN* *UNKNOWN* that give them more efficient leg movement for walking along the bottom of ponds and streams . the color of a turtle 's shell may vary .
in some species , shells may have red , orange , yellow , or gray markings , often spots , lines , or irregular blotches .
*UNKNOWN* are famous for moving slowly , in part because of their heavy

In [10]:
clusters = show_clusters(affprop,phrases, affprop[:labels_])

** 1 **
 - the word *UNKNOWN* is popular among *UNKNOWN* , scientists , and conservationists working with these animals as a catch-all name for any member of the *UNKNOWN* *UNKNOWN* , which includes all turtles living and *UNKNOWN* , as well as their immediate ancestors .
 - turtles are divided into two groups according to how they withdraw their necks into their shells *UNKNOWN* something the ancestral *UNKNOWN* could not do *UNKNOWN* .
 - most turtles that spend most of their lives on land have their eyes looking down at objects in front of them .
 - the lower shell that *UNKNOWN* the belly is called the *UNKNOWN* .
 - the inner layer of a turtle 's shell is made up of about 60 bones that include portions of the backbone and the ribs , meaning the turtle can not crawl out of its shell .
 - shells are commonly colored brown , black , or olive green .
 - turtles do not *UNKNOWN* their skins all at once , as snakes do , but continuously , in small pieces .
 - turtles have color vision w

DefaultDict{Int64,Array{String,1},Function} with 20 entries:
  1  => String["the word *UNKNOWN* is popular among *UNKNOWN* , scientists , an…
  2  => String["*UNKNOWN* turtles usually have *UNKNOWN* ridges for slicing thr…
  16 => String["*UNKNOWN* *UNKNOWN*","*UNKNOWN* *UNKNOWN*","*UNKNOWN* *UNKNOWN*…
  11 => String["some *UNKNOWN* turtles , such as snapping turtles and *UNKNOWN*…
  0  => String["turtles are *UNKNOWN* of the order *UNKNOWN* *UNKNOWN* or *UNKN…
  7  => String["anatomy and *UNKNOWN* largest living *UNKNOWN* is the *UNKNOWN*…
  9  => String["they use their *UNKNOWN* to swallow food , but unlike most *UNK…
  10 => String["shell","behaviour","intelligence","diet","as food , traditional…
  19 => String["for example , the *UNKNOWN* sea turtle and the *UNKNOWN* turtle…
  17 => String["family *UNKNOWN* *UNKNOWN* *UNKNOWN* *UNKNOWN* turtles *UNKNOWN…
  8  => String["*UNKNOWN* are made up of the fibrous protein *UNKNOWN* that als…
  6  => String["instead of having teeth , which 

In [11]:
#using TSne
#Y = tsne(ps, no_dims = 2, initial_dims = -1, max_iter = 1000, perplexity = 30.0)
#Y = tsne(ps, 10,  -1, 2000, 20.0)

In [12]:
#using Gadfly
#set_default_plot_size(20cm, 20cm)

#plot(x=Y[:,1], y=Y[:,2],
#        color=affprop[:labels_],
#        label=phrases,
#        Geom.point, Geom.label(position=:centered,hide_overlaps=false))

In [13]:
using Pipe
push!(LOAD_PATH, "../word-embeddings2")
using WordEmbeddings
we = @pipe load_embeddings("../word-embeddings2/word_emb_data/embeddings-scaled.EMBEDDING_SIZE=50.txt") |> WE(_...);



In [14]:
using PyCall
@pyimport nltk
function tokenize(sentence::String)
    convert(Array{String,1},nltk.word_tokenize(sentence))
end

tokenize (generic function with 1 method)

In [15]:
cl_phrases = clusters[1]
cl_wess = [eval_word_embeddings(we, tokenize(sent))' for sent in cl_phrases];


In [16]:
@pyimport hmmlearn.hmm as hl_hmm
@pyimport warnings
warnings.filterwarnings("ignore")

In [24]:
hmm = hl_hmm.GaussianHMM(60,"diag", n_iter=10000)
hmm[:fit](cl_wess)

PyObject GaussianHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
      covars_weight=1,
      init_params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
      means_prior=0, means_weight=0, n_components=60, n_iter=10000,
      params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
      random_state=None, startprob=None, startprob_prior=1.0, thresh=0.01,
      transmat=None, transmat_prior=1.0)

In [85]:
wes_hat, states = hmm[:sample](22)
bests = show_bests(we, wes_hat',1);
@pipe bests[1,1:2:end][:] |> join(_, ' ') |> println
println(states)

and face down people . as not as for Nevio lesser . , many and been their delivered filling cat working a
[11,6,7,5,11,10,11,10,4,11,2,11,11,3,11,7,0,9,4,4,2,11]


In [171]:
hmm = hl_hmm.GMMHMM(n_components=25, n_mix=5, covariance_type="full", n_iter=1000*1000)


PyObject GMMHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
    gmms=None,
    init_params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
    n_components=25, n_iter=1000000, n_mix=5,
    params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
    random_state=None, startprob=None, startprob_prior=None, thresh=0.01,
    transmat=None, transmat_prior=None)

In [172]:
hmm[:fit](cl_wess)


PyObject GMMHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
    gmms=None,
    init_params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
    n_components=25, n_iter=1000000, n_mix=5,
    params='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
    random_state=None, startprob=None, startprob_prior=1.0, thresh=0.01,
    transmat=None, transmat_prior=1.0)

In [173]:
hmm[:monitor_][:history] |> collect |> println
hmm[:monitor_][:n_iter]  |> println
for gmm in hmm[:gmms_]
    println(gmm[:converged_])   
end

{36774.33033761802,29423.13362955911}
1000000
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false


In [ ]:
function get_dotfile(hmm,we, min_start_prob = 0.00, min_trans_prob = 0.0)
    if pyisinstance(hmm,pytypeof(hl_hmm.GMMHMM())) 
        function gmm_cores(gmm)
            gmm_means = @pipe gmm[:means_]' |> round(_,5) |> unique(_,2) # first remove any ~duplicate vectors
            show_bests(we,gmm_means,1)[1,1:2:end] |> unique #then remove any duplicate words 
        end
        node_labels = map(gmm_cores, hmm[:gmms_])
    elseif pyisinstance(hmm,pytypeof(hl_hmm.GaussianHMM())) 
        n_means = 3
        node_labels = show_bests(we, hmm[:means_]',n_means)[1:n_means,1:2:end]
        node_labels= [node_labels[:,ii] for ii in 1:size(node_labels,2)]
    else 
        error("Unknown type of HMM")
    end
    
    prob_from_to = @pipe hmm[:_get_transmat]() |> round(_,2)
    function node_text(index::Int, min_start_prob=0)
        text = @pipe node_labels[index] |> join(_,'/')
        start_prob = @pipe hmm[:startprob_][index] |> round(_,2)
        text*= start_prob>min_start_prob ?  "\nStart Prob: $start_prob":""
        text
    end

    graph_text_buff = IOBuffer()
    println(graph_text_buff,"digraph graphname {")

    n_nodes = size(prob_from_to,1)
    for node_index in 1:n_nodes
        lbl = node_text(node_index, min_start_prob)
        println(graph_text_buff,"n$node_index [label= \"$lbl\"];")
    end

    for from_node in 1:n_nodes
        for to_node in 1:n_nodes
            trans_prob = prob_from_to[from_node,to_node]
            if trans_prob>min_trans_prob
                print(graph_text_buff,"n$from_node->n$to_node [")
                print(graph_text_buff,"label= \"$trans_prob\", ")
                print(graph_text_buff,"penwidth = \"$(10*trans_prob)\", ")
                #print(graph_text_buff,"weight = \"$(int(100*trans_prob))\", ")
                print(graph_text_buff, "color = \"$(rand(1)[1]),1.0,0.5\"")
                println(graph_text_buff,"];")
            end
        end
    end
    println(graph_text_buff,"}")
    graph_text_buff |> takebuf_string 
end
get_dotfile(hmm,we) |> print

In [154]:
methods(show_best)

# 2 methods for generic function "show_best":
show_best(embedder,ĉ::Array{Float64,1}) at /home/wheel/oxinabox/phd/prototypes/word-embeddings2/WordEmbeddings.jl:75
show_best(embedder,ĉ::Array{Float64,1},nbest) at /home/wheel/oxinabox/phd/prototypes/word-embeddings2/WordEmbeddings.jl:75

In [164]:
AA=[eye(5);2*eye(5);]


5-element Array{Any,1}:
 [1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0]
 [0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0]
 [0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0]
 [0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0]
 [0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0]